In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

In [2]:
# from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
import seaborn as sns
import random
from ta import add_all_ta_features
from ta.utils import dropna
from ta.trend import *

In [3]:
##### Initialization of global variables here ###

order_books = defaultdict(set)
df_ratio = pd.read_csv("ratio_dict.csv", index_col = 0).set_index("Pair")
df_global = pd.DataFrame({"date":[]})


country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "China":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}

NameError: name 'defaultdict' is not defined

In [13]:
adr_symbol = "CAJ"
adr_exchange = 'SMART'
adr_currency = "USD"

underlying_symbol = '7751'
underlying_exchange = 'TSEJ'
underlying_currency = 'JPY'
underlying_country = "Japan"

adr_contract = Contract(symbol = adr_symbol, secType = 'STK', exchange = adr_exchange, currency = adr_currency)
ib.qualifyContracts(adr_contract)
underlying_contract = Contract(symbol = underlying_symbol, secType = 'STK', exchange = underlying_exchange, currency = underlying_currency)
ib.qualifyContracts(underlying_contract)

history = '60 S'
freq = '1 min'
side ='Trades'

df_test = get_data(adr_contract, history, freq, side)


In [14]:
df_test

,date,open,high,low,close,volume,average,barCount
0,2021-04-21 15:59:00,23.43,23.44,23.42,23.44,27,23.432,20


In [6]:
#define a function to get IB data. endDate is the last date of the historical data 

def get_data(contract, history, freq, side, endDate =''):
    bar = ib.reqHistoricalData(
        contract,
        endDateTime = endDate,
        durationStr=history,
        barSizeSetting= freq,
        whatToShow=side,
        useRTH=True,
        formatDate=1)
    return util.df(bar)

history = '300 S'
freq = '1 min'
side ='Trades'

# write getter functions?

In [41]:
# Initialization of hard coded data

aus_adr_underlying_pairs = [
    ["ATHE", "ATH"],["GENE", "GTG"],["IMMP", "IMM"],["IMRN","IMC"],["JHX", "JHX"],["KZIA", "KZA"],
    ["MESO","MSB"],["PLL", "PLL"],["WBK","WBC"]
]

hk_adr_underlying_pairs = [
    ["ACH", "2600"],["BGNE", "6160"],["CEA","670"],["HNP", "902"],["LFC", "2628"],["PTR", "857"],
    ["SHI", "338"],["SNP","386"],["ZNH", "1055"]
]

jpn_adr_underlying_pairs = [
    ["CAJ", "7751"],["HMC","7267"],["IX","8591"],["MFG","8411"],["MUFG","8306"],["NMR","8604"],
    ["SMFG","8316"],["SONY","6758"],["TAK", "4502"],["TM", "7203"]
]

country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "HK":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}

forex_pairs = [("AUD", "USD", AUS_opening),  ("USD", "HKD", HK_opening), 
               ("USD","JPY", JPN_opening)]


# functions return the market opening time adjusted for DST

us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    } 

def next_market_open_day(date_now):
    '''
    Returns today if Mon - Thurs, returns next Mon if Fri thru Sun
    '''
    while date_now.weekday()>4:
        date_now += timedelta(days = 1)
    return date_now

def AUS_opening(date_now):
    date_now = next_market_open_day(date_now)
    us_start, us_end = us_dst[date_now.year]
    if us_start < date_now and date_now < us_end:
        return datetime.combine(date_now,time(10,0)) - timedelta(hours = 14)
    else:
        return datetime.combine(date_now, time(10,0)) - timedelta(hours = 15)

def AUS_closing(date_now):
    open_time = AUS_opening(date_now)
    return open_time + timedelta(hours = 6)

def HK_opening(date_now):
    date_now = next_market_open_day(date_now)
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,30)) - timedelta(hours = 12)
    else:
        return datetime.combine(date_now, time(9,30)) - timedelta(hours = 13)
    
def HK_closing(date_now):
    open_time = HK_opening(date_now)
    return open_time + timedelta(hours = 6.5)
    
def JPN_opening(date_now):
    date_now = next_market_open_day(date_now)
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 13)
    else:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 14)
    
def JPN_closing(date_now):
    open_time = JPN_opening(date_now)
    return open_time + timedelta(hours = 6)

def US_opening(date_now):
    return datetime.combine(next_market_open_day(date_now),time(9,30))

def US_closing(date_now):
    return datetime.combine(next_market_open_day(date_now),time(16,0))

In [71]:
country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "HK":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}


# Store all the historical adr/underlying open and close data

df_global = pd.DataFrame({"date":[]})

def pull_asian_close(days = 1):
    global df_global, country_info

    today = datetime.now().date()
    history = f"{days} D"
    freq = "1 Day"
    side = 'Trades'
    
    df_asian_close = pd.DataFrame({"date":[]})
    for key in country_info.keys():
        pairs = country_info[key]["Pairs"]
        exchange = country_info[key]["Exchange"]
        currency = country_info[key]["Currency"]
        
        for p in pairs:
            underlying = p[1]
            contract = Contract(symbol = underlying, secType = 'STK', exchange = exchange, currency = currency)
            try:
                df = get_data(contract, history, freq, side, endDate =today)
                df = df.loc[:, ["date", "close"]]
                col_name = "underlying_" + underlying + "_close"
                df = df.rename(columns = {"close":col_name})
#                 print(df)
                df_asian_close = pd.merge(df, df_asian_close, how = "outer")
            except:
                pass
            
    df_global = pd.concat([df_global, df_asian_close])

    
    
def pull_us_close(days = 1):
    global df_global, country_info

    today = datetime.now().date()
    history = f"{days} D"
    freq = "1 Day"
    side = 'Trades'
    
    df_us_close = pd.DataFrame({"date":[]})
    for key in country_info.keys():
        pairs = country_info[key]["Pairs"]
        exchange = "SMART"
        currency = "USD"
        
        for p in pairs:
            underlying = p[0] # get the ticker of the ADR
            contract = Contract(symbol = underlying, secType = 'STK', exchange = exchange, currency = currency)
            df = get_data(contract, history, freq, side, endDate ='')
            df = df.loc[:, ["date", "close"]]
            col_name = "adr_" + underlying + "_close"
            df = df.rename(columns = {"close":col_name})
            df_us_close = pd.merge(df, df_us_close, how = 'outer')
            
    df_global = pd.merge(df_global, df_us_close)

df_forex = pd.DataFrame({"date":[]})
def pull_forex_data(history = "1 D", freq = "1 min", side = "BID_ASK", end_date = "" ):
    '''
    Updates all three forex pairs
    '''
    
    global df_global, country_info, forex_pairs, test


    for currency1, currency2, * in forex_pairs:
        contract = Contract(symbol = currency1, secType = 'CASH', exchange = "IDEALPRO", currency = currency2)
        df = get_data(contract, history, freq, side)

        if currency2 == "USD":
            df["mid"] = (df["open"] + df["close"])/2
            forex_string = f"{currency1}.{currency2}"
        else:
            df["mid"] = 2/(df["open"] + df["close"])
            forex_string = f"{currency2}.{currency1}"

        df = df.loc[:, ["date", "MID"]]
        df_us_open = df[[(minute == US_opening(minute)) for minute in df["date"]]]

        df_asian_open = df[[(minute == asian_opening(minute)) for minute in df["date"]]]


        df_us_open = df_us_open.rename(columns = {"close":f"{forex_string}_MID_US_open"})
        df_asian_open = df_asian_open.rename(columns = {"close":f"{forex_string}_MID_{currency2}_open"})

        df_forex = pd.merge(df_forex, df_us_open, how = "outer")
        df_forex = pd.merge(df_forex, df_asian_open, how = "outer")

        df_global = pd.concat([df_global, df_forex])
        df_global.to_csv("global_df.csv")


In [ ]:
# def pull_forex_data(history = "1 D", freq = "1 min", side = "BID_ASK", end_date = "" ):
#     '''
#     Updates all three forex pairs
#     '''
    
#     global df_global, country_info, forex_pairs, test

    
#     df_forex = pd.DataFrame({"date":[]})
#     for currency1, currency2, asian_opening in forex_pairs:
#         contract = Contract(symbol = currency1, secType = 'CASH', exchange = "IDEALPRO", currency = currency2)
#         df = get_data(contract, history, freq, side)

#         if currency2 == "USD":
#             df["mid"] = (df["open"] + df["close"])/2
#             forex_string = f"{currency1}.{currency2}"
#         else:
#             df["mid"] = 2/(df["open"] + df["close"])
#             forex_string = f"{currency2}.{currency1}"

#         df = df.loc[:, ["date", "MID"]]
#         df_us_open = df[[(minute == US_opening(minute)) for minute in df["date"]]]

#         df_asian_open = df[[(minute == asian_opening(minute)) for minute in df["date"]]]


#         df_us_open = df_us_open.rename(columns = {"close":f"{forex_string}_MID_US_open"})
#         df_asian_open = df_asian_open.rename(columns = {"close":f"{forex_string}_MID_{currency2}_open"})

#         df_forex = pd.merge(df_forex, df_us_open, how = "outer")
#         df_forex = pd.merge(df_forex, df_asian_open, how = "outer")

#         df_global = pd.concat([df_global, df_forex])
#         df_global.to_csv("global_df.csv")

In [4]:
# pull_asian_close(10)
# pull_us_close(10)
# pull_forex_data()

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.


In [28]:

def z_score()

In [18]:
currency1 = "AUD"
secType = "CASH"
currency2 = "USD"
exchange = "IDEALPRO"

history = "1 D"
freq = "1 min"
side = "BID_ASK"

forex_contract = Contract(symbol = currency1, secType = secType, exchange = exchange, currency = currency2)
ib.qualifyContracts(forex_contract)
df_forex = get_data(forex_contract, history, freq, side)


In [ ]:
# 8.30 am
    # Pull Historical Forex Data (Minute, for lookback window)
    # Pull Close Data for Asian Market at 9am (Before US Market opens)
# 9.28 am 
    # Pull Forex Data for three currencies (For Z score)
    # Calculate Z Score 
    # for each pair:
        # if invested:
            # if risk condition or exit condition:
                # liquidate
            # else:
                # continue
        # if entry condition:
            # determine order size from risk metric and ADT over last 5 days
            # if 0: no order made
            # else: 
                # Define contract
                # make order (What order? Market Order?)


# 6 pm
    # Pull Close Data for US Market at 6pm (Before Asian Market opens)
    # Pull Historical Forex Data

# 2 min before Asian Market Open
    # Pull Forex Data
    # Calculate Z score using market close and minute forex data

In [34]:
# initialize data lists, hyperparameters etc


path_data = "live_trading_data/"
df_forex = pd.DataFrame
def initialize_forex():
    global df_forex
    
    now = datetime.now()
    pull_head = now-timedelta(days = round(7/5*lookback_window))

    pull_again = False
    for currency1, currency2 in forex_pairs:
        forex_string = currency1 + "." + currency2
        df_forex[forex_string] = pd.read_csv(path + forex_string + ".csv")
        # check if forex information is updated:
        if df_forex["date"][0].date() < datetime.now().date():
            pull_again = True
            pull_head = max(df_forex["date"][0], pull_head)
            break

    if pull_again:
        now = datetime.now()
        while pull_head < now:
            print(pull_head.date())
            pull_forex_data(df_forex, history = "1 D", freq = "1 min", side = "BID_ASK", end_date = pull_head)
            pull_head += timedelta(days = 1)
        for currency1, currency2 in forex_pairs:
            forex_string = currency1 + "." + currency2
            df_forex[forex_string].to_csv(path + forex_string + ".csv")
    else:
        print("df_forex initialized")


Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.


In [ ]:
country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "HK":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}


live_data = {}
def pull_live_data():
    global live_data, country_info, forex_pairs
    
    dict_forex = {}
    for country in forex_pairs:
        currency1, currency2, * = country

        history = "60 S"
        freq = "1 min"
        side = "BID_ASK"

        forex_contract = Contract(symbol = currency1, secType = "CASH", exchange = "IDEALPRO", currency = currency2)
        ib.qualifyContracts(forex_contract)
        df_temp = get_data(forex_contract, history, freq, side)
        
        latest_tick = df_temp.iloc[-1]
        if currency2 == "USD":
            live_data[f"{currency1}.{currency2}"] = (latest_tick["open"] + latest_tick["close"])/2

        else:
            live_data[f"{currency2}.{currency2}"] = 2/(latest_tick["open"] + latest_tick["close"])
            
    for country in country_info.keys():
        currency = country_info[country]["Currency"]
        exchange = country_info[country]["Exchange"]
        pairs_list = country_info[country]["Pairs"]
        
        side = "Trades"

        for ticker_ADR, ticker_underlying in pairs_list:
            contract = Contract(symbol = ticker_ADR, secType = "STK", exchange = "SMART", currency = currency)
            ib.qualifyContracts(contract)
            df_temp = get_data(contract, history, freq, side)
            latest_tick = df_temp.iloc[-1]
            live_data[ticker_ADR] = latest_tick['close'] # NEED TO MULTIPLY BY ADR RATIO
            
            
            contract = Contract(symbol = ticker_underlying, secType = "STK", exchange = exchange, currency = currency)
            ib.qualifyContracts(contract)
            df_temp = get_data(contract, history, freq, side)
            latest_tick = df_temp.iloc[-1]
            if country == "Australia":
                live_data[ticker_ADR] = latest_tick['close'] * live_data["AUD.USD"] # NEED TO MULTIPLY BY ADR RATIO
            elif country == "Japan":
                live_data[ticker_ADR] = latest_tick['close'] * live_data["JPY.USD"] # NEED TO MULTIPLY BY ADR RATIO
            elif country == "HK":
                live_data[ticker_ADR] = latest_tick['close'] * live_data["HKD.USD"] # NEED TO MULTIPLY BY ADR RATIO
        
    
    
def intraday_trader(HK, AUS, JPN, US):
    
    

In [ ]:
def z_score

In [ ]:
# update date
while True:
    # keep track of what day it is
    today = datetime.date() # ideally the algorithm starts at 8am in the morning
    
    # keep track of how many times we pull data for calibration
    bef_us_pull = False
    bef_us_zscore = False
    bef_asia_pull = False
    bef_jpn_zscore = False
    bef_aus_zscore = False
    bef_hk_zscore = False
    
    # Various Open and Close hours for various markets
    
    US_open = US_opening(today)
    US_close = US_closing(today)
    HK_open = HK_opening(today + 1)
    HK_close = HK_closing(today + 1)
    JPN_open = JPN_opening(today + 1)
    JPN_close = JPN_closing(today + 1)
    AUS_open = AUS_opening(today + 1)
    AUS_close = AUS_closing(today + 1)
    
    # Keep track if pair is open
    
    while now < US_close and now < HK_close and now < JPN_close and now < AUS_close:
        HK_trading = False
        AUS_trading = False
        US_trading = False
        JPN_trading = False
        
        # Before US Market Open pull historical data
        if not bef_us_pull and now > US_open - timedelta(hours = 1):
            bef_us_pull = True

            # Pull asian close for underlying stocks
            pull_asian_close()
        
        # Right before US Market Open for Z Score
        if not bef_us_zscore and now > US_open - timedelta(minutes = 4):
            bef_us_zscore = True
            pull_forex_data(df_forex, history = "21600 seconds")
            # Calculate Z score and make decision on trade and size
            
        if US_open < now and now < US_close:
            # Pull minute data to check whether to liquidate positions
            # Check pending limit orders
            US_trading = True
            
        
        # before asian markets open, pull us market close
        if not bef_asia_pull and now > US_close + timedelta(hours = 1):
            bef_asia_pull = True
            # pull us close
        
        # before Hong Kong market open, pull forex to calculate Z score
        if not bef_hk_zscore and now > HK_open - timedelta(minutes = 4):
            bef_hk_zscore = True
            pull_forex_data(df_forex, history = "21600 seconds")
            # calculate Z score and make decision on trade and size
        
        # check on Hong Kong Stocks
        if HK_open < now and now < HK_close:
            # Pull minute data to check whether to liquidate positions
            HK_trading = True
            # Check pending limit orders
        
        # before Japan market open, pull forex to calculate Z score
        if not bef_jpn_zscore and now > JPN_opening(today) - timedelta(minutes = 4):
            bef_JPN_zscore = True
            pull_forex_data(df_forex, history = "21600 seconds")
            # calculate Z score and make decision on trade and size
        
        # check on Japan stocks
        if JPN_open < now and now < JPN_close:
            # Pull minute data to check whether to liquidate positions
            JPN_trading = True
            # Check pending limit orders
            
        # before Australia market open, pull forex to calculate Z score            
        if not bef_aus_zscore and now > AUS_opening(today) - timedelta(minutes = 4):
            bef_aus_zscore = True
            pull_forex_data(df_forex, history = "21600 seconds")
        
        # check on Australian Stocks
        if AUS_open < now and now < AUS_close:
            # Pull minute data to check whether to liquidate positions
            AUS_trading = True
            # Check pending limit orders         
        

        ib.sleep(120) # Loop every two minutes
        now = datetime.now()

[Contract(secType='STK', conId=13905840, symbol='7751', exchange='TSEJ', primaryExchange='TSEJ', currency='JPY', localSymbol='7751.T', tradingClass='7751')]